In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
import random
import matplotlib.pyplot as plt

COLOR_TO_CLASS = {
    (230, 25, 75): 0,
    (145, 30, 180): 1,
    (60, 180, 75): 2,
    (245, 130, 48): 3,
    (255, 255, 255): 4,
    (0, 130, 200): 5,
    (255, 0, 255): 6
}

CLASS_TO_COLOR = {
    0: (230, 25, 75),
    1: (145, 30, 180),
    2: (60, 180, 75),
    3: (245, 130, 48),
    4: (255, 255, 255),
    5: (0, 130, 200),
}

IGNORE_COLOR = (255, 0, 255)
NUM_CLASSES = 6
CLASS_NAMES = ['Building', 'Clutter', 'Vegetation', 'Water', 'Background', 'Car']


def load_image_paths(df, image_dir, elevation_dir, slope_dir, label_dir):
    image_paths = df["tile_id"].apply(lambda tid: os.path.join(image_dir, f"{tid}-ortho.png")).tolist()
    elevation_paths = df["tile_id"].apply(lambda tid: os.path.join(elevation_dir, f"{tid}-elev.npy")).tolist()
    slope_paths = df["tile_id"].apply(lambda tid: os.path.join(slope_dir, f"{tid}-slope.npy")).tolist()
    label_paths = df["tile_id"].apply(lambda tid: os.path.join(label_dir, f"{tid}-label.png")).tolist()
    tile_ids = df["tile_id"].tolist()
    return image_paths, elevation_paths, slope_paths, label_paths, tile_ids


def augment_image(rgb, elev, slope, label):
    label = tf.expand_dims(label, axis=-1) if tf.rank(label) == 2 else label
    if tf.random.uniform([]) > 0.5:
        rgb = tf.image.flip_left_right(rgb)
        elev = tf.image.flip_left_right(elev)
        slope = tf.image.flip_left_right(slope)
        label = tf.image.flip_left_right(label)
    if tf.random.uniform([]) > 0.5:
        rgb = tf.image.flip_up_down(rgb)
        elev = tf.image.flip_up_down(elev)
        slope = tf.image.flip_up_down(slope)
        label = tf.image.flip_up_down(label)
    label = tf.squeeze(label, axis=-1)
    return rgb, elev, slope, label


def decode_coloured_label(label_rgb):
    label_rgb = tf.cast(label_rgb, tf.uint8)
    label_flat = tf.reshape(label_rgb, [-1, 3])
    colors = tf.constant(list(COLOR_TO_CLASS.keys()), dtype=tf.uint8)
    class_ids = tf.constant(list(COLOR_TO_CLASS.values()), dtype=tf.int32)
    mask = tf.reduce_all(tf.equal(tf.expand_dims(label_flat, 1), colors), axis=2)
    indices = tf.argmax(tf.cast(mask, tf.int32), axis=1)
    mapped = tf.gather(class_ids, indices)
    return tf.reshape(mapped, [tf.shape(label_rgb)[0], tf.shape(label_rgb)[1]])


def _load_npy(path):
    return np.load(path.decode("utf-8")).astype(np.float32)


def parse_elevation(rgb_path, elev_path, slope_path, label_path, tile_id,
                    split='train', augment=False, tile_size=256, dummy=False):
    rgb = tf.io.read_file(rgb_path)
    rgb = tf.image.decode_png(rgb, channels=3)
    rgb = tf.image.convert_image_dtype(rgb, tf.float32)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=3)
    label = decode_coloured_label(label)

    if dummy:
        elev = tf.random.uniform(tf.shape(rgb)[..., :1])
        slope = tf.random.uniform(tf.shape(rgb)[..., :1])
    else:
        elev = tf.numpy_function(_load_npy, [elev_path], tf.float32)
        slope = tf.numpy_function(_load_npy, [slope_path], tf.float32)
        elev.set_shape([None, None, 1])
        slope.set_shape([None, None, 1])

    if split == 'train' and augment:
        rgb, elev, slope, label = augment_image(rgb, elev, slope, label)

    label.set_shape([None, None])
    label = tf.image.resize(label[..., tf.newaxis], [tile_size, tile_size], method='nearest')
    label = tf.squeeze(label, axis=-1)
    label = tf.cast(label, tf.int32)
    label_onehot = tf.one_hot(label, depth=NUM_CLASSES)

    rgb = tf.image.resize(rgb, [tile_size, tile_size])
    elev = tf.image.resize(elev, [tile_size, tile_size])
    slope = tf.image.resize(slope, [tile_size, tile_size])
    input_image = tf.concat([rgb, elev, slope], axis=-1)

    return input_image, label_onehot


def parse_tile(rgb_path, label_path, tile_id, split='train', augment=False, tile_size=256):
    rgb = tf.io.read_file(rgb_path)
    rgb = tf.image.decode_png(rgb, channels=3)
    rgb = tf.image.convert_image_dtype(rgb, tf.float32)

    label = tf.io.read_file(label_path)
    label = tf.image.decode_png(label, channels=3)
    label = decode_coloured_label(label)

    if split == 'train' and augment:
        dummy_elev = tf.zeros_like(rgb[..., :1])
        dummy_slope = tf.zeros_like(rgb[..., :1])
        rgb, dummy_elev, dummy_slope, label = augment_image(rgb, dummy_elev, dummy_slope, label)

    rgb.set_shape([None, None, 3])
    label.set_shape([None, None])

    rgb = tf.image.resize(rgb, [tile_size, tile_size])
    label = tf.image.resize(label[..., tf.newaxis], [tile_size, tile_size], method='nearest')
    label = tf.squeeze(label, axis=-1)
    label = tf.cast(label, tf.int32)
    label_onehot = tf.one_hot(label, depth=NUM_CLASSES)

    return rgb, label_onehot


def build_tf_dataset(df, image_dir, elevation_dir, slope_dir, label_dir,
                     input_type='rgb', batch_size=32, split='train',
                     augment=False, shuffle=True, tile_size=256, dummy=False):

    image_paths, elevation_paths, slope_paths, label_paths, tile_ids = load_image_paths(df, image_dir, elevation_dir, slope_dir, label_dir)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, elevation_paths, slope_paths, label_paths, tile_ids))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths), reshuffle_each_iteration=(split == 'train'))
        target_boost_classes=[3, 5]

    def class_boost_selector(rgb_path, elev_path, slope_path, label_path, tile_id):
        tile_id_str = tile_id.numpy().decode("utf-8")
        if tile_id_str not in tile_ids:
            return False
        ratios = np.array([tile_ids[tile_id_str].get(f"{i}: {cls}", 0.0) for i, cls in enumerate(CLASS_NAMES)])
        weights = np.array([3.0 if i in target_boost_classes else 1.0 for i in range(NUM_CLASSES)])
        weighted_presence = np.sum(ratios * weights)
        return np.random.rand() < weighted_presence

    if split == 'train' and target_boost_classes:
        dataset = dataset.filter(lambda rgb, elev, slope, label, tile_id:
                                 tf.py_function(func=class_boost_selector,
                                                inp=[rgb, elev, slope, label, tile_id],
                                                Tout=tf.bool))

    def map_fn(rgb_path, elev_path, slope_path, label_path, tile_id):
        if input_type == 'rgb':
            return parse_tile(rgb_path, label_path, tile_id, split, augment, tile_size)
        elif input_type == 'rgb_elev':
            return parse_elevation(rgb_path, elev_path, slope_path, label_path, tile_id, split, augment, tile_size, dummy=dummy)
        else:
            raise ValueError(f"Unsupported input_type: {input_type}")

    dataset = dataset.map(map_fn, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset
